In [1]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import gc, math
import pickle

from keras.models import Sequential
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import Model, load_model, model_from_json

from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import backend as K
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

from cnnmodels import vgg_std16_model, preprocess_input, create_rect5, load_img, train_generator, test_generator
from cnnmodels import identity_block, testcv_generator, conv_block, resnet50_model

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [176]:
# Params
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
ROWS, COLS = 224, 224
CHECKPOINT_DIR = 'log/checkpoint09/'
BATCHSIZE = 32
CERV_CLASSES = ['Type_1', 'Type_2', 'Type_3']
nb_perClass = int(BATCHSIZE / len(CERV_CLASSES))
TRAIN_DIR = '../data/original/train'
TEST_DIR = '../data/original/test'
DATA_DIR = '../data/original'
num_class = len(CERV_CLASSES)
full = True
bags = 5

In [218]:
def train_generator(datagen, df):
    while 1:
        batch_x = np.zeros((BATCHSIZE, ROWS, COLS, 3), dtype=K.floatx())
        batch_y = np.zeros((BATCHSIZE, len(CERV_CLASSES)), dtype=K.floatx())
        fn = lambda obj: obj.loc[np.random.choice(obj.index, size=nb_perClass, replace=False),:]
        batch_df = df.groupby('class', as_index=True).apply(fn)
        i = 0
        for index,row in batch_df.iterrows():
            row = row.tolist()
            image_file = row[0]
            typ_class = row[1]
            img = Image.open(image_file).resize((ROWS, COLS))
            img = img.convert('RGB')
            x = np.asarray(img, dtype=K.floatx())
            #x = datagen.random_transform(x)
            x = preprocess_input(x)
            batch_x[i] = x
            batch_y[i,CERV_CLASSES.index(typ_class)] = 1
            i += 1
        #return (batch_x, batch_y)
        yield (batch_x.transpose(0, 3, 1, 2), batch_y)

In [4]:
def test_generator(df, datagen, batch_size = BATCHSIZE):
    n = df.shape[0]
    batch_index = 0
    while 1:
        current_index = batch_index * batch_size
        if n >= current_index + batch_size:
            current_batch_size = batch_size
            batch_index += 1    
        else:
            current_batch_size = n - current_index
            batch_index = 0        
        batch_df = df[current_index:current_index+current_batch_size]
        batch_x = np.zeros((batch_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
        i = 0
        for index,row in batch_df.iterrows():
            row = row.tolist()
            image_file = row[0]
            # typ_class = row[1]
            img = Image.open(image_file).resize((ROWS, COLS))
            img = img.convert('RGB')
            x = np.asarray(img, dtype=K.floatx())
            # x = datagen.random_transform(x)
            x = preprocess_input(x)
            batch_x[i] = x
            i += 1
        if batch_index%100 == 0: print(batch_index)
        # return (batch_x.transpose(0, 3, 1, 2))
        yield(batch_x.transpose(0, 3, 1, 2))

In [5]:
train_datagen = ImageDataGenerator(
    rotation_range=180,
    shear_range=0.2,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True)

In [6]:
img_ls = []
y_ls = []
imgadd_ls = []
yadd_ls = []
for typ in CERV_CLASSES:
    for img in os.listdir(os.path.join(TRAIN_DIR, typ)):
        if img != '.DS_Store':
            img_ls.append(os.path.join(TRAIN_DIR, typ, img))
            y_ls.append(typ)
for typ in CERV_CLASSES:
    for img in os.listdir(os.path.join(DATA_DIR, typ)):
        if img != '.DS_Store':
            imgadd_ls.append(os.path.join(DATA_DIR, typ, img))
            yadd_ls.append(typ)
train_orig_all  = pd.DataFrame({'class': y_ls, 'img': img_ls, })[['img', 'class']]
train_addl_all  = pd.DataFrame({'class': yadd_ls, 'img': imgadd_ls, })[['img', 'class']]
print(train_orig_all.shape)
print(train_addl_all.shape)

(1480, 2)
(6729, 2)


In [7]:
train_all = train_orig_all
test_df =   train_addl_all

In [8]:
# Split into train and valid
valid_set = pd.read_csv("../val_images.csv", header = None, names = ['img']).img.tolist()
valid_set[-4:]

['Type_2/3498.jpg', 'Type_2/1341.jpg', 'Type_3/6017.jpg', 'Type_2/5629.jpg']

In [10]:
valid_df = train_all[train_all['img'].str.replace('../data/original/', '').isin(valid_set)]
if full == True:
    train_df = train_all
else:
    train_df = train_all[~train_all['img'].str.replace('../data/original/', '').isin(valid_set)]
samples_per_epoch=BATCHSIZE*math.ceil(train_df.groupby('class').size()['Type_2']/nb_perClass)
print(train_df.shape)
print(valid_df.shape)

(1480, 2)
(242, 2)


In [11]:
# Make our validation set
l = valid_df.groupby('class').size()
valid_x = np.zeros((valid_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
valid_y = np.zeros((valid_df.shape[0], len(CERV_CLASSES)), dtype=K.floatx())
i = 0
for index,row in valid_df.iterrows():
    row = row.tolist()
    image_file = row[0]
    typ_class = row[1]
    img = Image.open(image_file).resize((ROWS, COLS))
    img = img.convert('RGB')
    x = np.asarray(img, dtype=K.floatx())
    # x = datagen.random_transform(x)
    x = preprocess_input(x)
    valid_x[i] = x
    valid_y[i,CERV_CLASSES.index(typ_class)] = 1
    i += 1
valid_x = valid_x.transpose(0, 3, 1, 2)

In [12]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')        
model_checkpoint = ModelCheckpoint(filepath=CHECKPOINT_DIR+'weights.{epoch:03d}-{val_loss:.4f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
# learningrate_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', epsilon=0.001, cooldown=0, min_lr=0)
# tensorboard = TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=False, write_images=True)

In [13]:
print "Model creation... "
nb_epoch = 4
model = resnet50_model(ROWS, COLS, channel, num_class)
for layer in model.layers:
    layer.trainable = False
for layer in model.layers[-3:]:
    layer.trainable = True

# Start Fine-tuning
print "Fine tune part 1"
model.fit_generator(train_generator(train_datagen, train_df),
          nb_epoch=nb_epoch,
          samples_per_epoch=samples_per_epoch, #50000,
          verbose=1,
          validation_data=(valid_x, valid_y),
          callbacks=[early_stopping, model_checkpoint],
          )

Model creation... 
Fine tune part 1
Epoch 1/4
2496/2496 [==============================] - 138s - loss: 1.0365 - acc: 0.4856 - val_loss: 0.9778 - val_acc: 0.4835
Epoch 2/4
2496/2496 [==============================] - 136s - loss: 0.7944 - acc: 0.6446 - val_loss: 0.8298 - val_acc: 0.5702
Epoch 3/4
2496/2496 [==============================] - 136s - loss: 0.6738 - acc: 0.7272 - val_loss: 0.6766 - val_acc: 0.7231
Epoch 4/4
2496/2496 [==============================] - 136s - loss: 0.5764 - acc: 0.7993 - val_loss: 0.5472 - val_acc: 0.8264


In [14]:
for layer in model.layers[38:]:
    layer.trainable = True
#model.optimizer.lr = 1e-5
nb_epoch = 2
print "Fine tune part 2"
model.fit_generator(train_generator(train_datagen, df=train_df),
          nb_epoch=nb_epoch,
          samples_per_epoch=samples_per_epoch,
          verbose=1,
          validation_data=(valid_x, valid_y),
          callbacks=[model_checkpoint, early_stopping], # , 
          )

Fine tune part 2
Epoch 1/2
2496/2496 [==============================] - 136s - loss: 0.4908 - acc: 0.8534 - val_loss: 0.4567 - val_acc: 0.8760
Epoch 2/2
2496/2496 [==============================] - 136s - loss: 0.4293 - acc: 0.8822 - val_loss: 0.3910 - val_acc: 0.9050


In [18]:
test_preds_ls = []
for ii in range(2):
    test_preds_ls.append(model.predict_generator(test_generator(test_df, train_datagen), 
                                         val_samples = test_df.shape[0])) 
    model.fit_generator(train_generator(train_datagen, df=train_df),
              nb_epoch=1,
              samples_per_epoch=samples_per_epoch,
              verbose=1,
              validation_data=(valid_x, valid_y),
              #callbacks=[model_checkpoint, early_stopping], # , 
              )


100
200
0
Epoch 1/1
2496/2496 [==============================] - 135s - loss: 0.3269 - acc: 0.9191 - val_loss: 0.2847 - val_acc: 0.9587
100
200
0
Epoch 1/1
2496/2496 [==============================] - 136s - loss: 0.2694 - acc: 0.9407 - val_loss: 0.2453 - val_acc: 0.9587


In [20]:
test_preds_ls.append(model.predict_generator(test_generator(test_df, train_datagen), 
                                         val_samples = test_df.shape[0])) 

100
200
0


In [25]:
test_preds = sum(test_preds_ls)/len(test_preds_ls)
test_sub = pd.DataFrame(test_preds, columns=CERV_CLASSES)
test_sub['image_name'] = test_df['img']#.str.split('/').apply(lambda x: x[-1])
test_sub = test_sub[['image_name'] + CERV_CLASSES ]
test_sub.to_csv('../feat/additional_pred.csv', index=False)#, compression='gzip')
test_sub.head(3)

,image_name,Type_1,Type_2,Type_3
0,../data/original/Type_1/2399.jpg,0.564977,0.412536,0.022487
1,../data/original/Type_1/1704.jpg,0.762905,0.157422,0.079673
2,../data/original/Type_1/3237.jpg,0.825580,0.159832,0.014588


In [51]:
good_additional = []
bad_additional = []
for c, row in test_sub.iterrows():
    typ = int(row[0].split('/')[3][-1])
    if row[typ] >= 0.2:
        good_additional.append(row[0])
    else:
        bad_additional.append(row[0])

In [52]:
print(len(good_additional), len(bad_additional))

(5378, 1351)


### Now that we have the good and bad additionals, lets do a full run

In [69]:
train_all = pd.concat([train_orig_all, train_addl_all])

In [70]:
train_all = train_all[~train_all['img'].isin(bad_additional)]

In [72]:
img_ls = []
for img in os.listdir(TEST_DIR):
    if img != '.DS_Store':
        img_ls.append(os.path.join(TEST_DIR, img))
test_df  = pd.DataFrame({'img': img_ls}) 

In [73]:
train_all.tail(3)

,img,class
6726,../data/original/Type_3/5391.jpg,Type_3
6727,../data/original/Type_3/4116.jpg,Type_3
6728,../data/original/Type_3/568.jpg,Type_3


In [74]:
# Split into train and valid
valid_set = pd.read_csv("../val_images.csv", header = None, names = ['img']).img.tolist()
valid_set[-4:]

['Type_2/3498.jpg', 'Type_2/1341.jpg', 'Type_3/6017.jpg', 'Type_2/5629.jpg']

In [75]:
valid_df = train_all[train_all['img'].str.replace('../data/original/', '').isin(valid_set)]
if full == True:
    train_df = train_all
else:
    train_df = train_all[~train_all['img'].str.replace('../data/original/', '').isin(valid_set)]
samples_per_epoch=BATCHSIZE*math.ceil(train_df.groupby('class').size()['Type_2']/nb_perClass)
print(train_df.shape)
print(valid_df.shape)

(6858, 2)
(1368, 2)


In [220]:
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [221]:
# Make our validation set
l = valid_df.groupby('class').size()
valid_x = np.zeros((valid_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
valid_y = np.zeros((valid_df.shape[0], len(CERV_CLASSES)), dtype=K.floatx())
i = 0
for index,row in valid_df.iterrows():
    row = row.tolist()
    image_file = row[0]
    typ_class = row[1]
    img = Image.open(image_file).resize((ROWS, COLS))
    img = img.convert('RGB')
    x = np.asarray(img, dtype=K.floatx())
    # x = datagen.random_transform(x)
    x = preprocess_input(x)
    valid_x[i] = x
    valid_y[i,CERV_CLASSES.index(typ_class)] = 1
    i += 1
valid_x = valid_x.transpose(0, 3, 1, 2)

In [222]:
print "Model creation... "
nb_epoch = 6
model = resnet50_model(ROWS, COLS, channel, num_class)
for layer in model.layers:
    layer.trainable = False
for layer in model.layers[-3:]:
    layer.trainable = True

# Start Fine-tuning
print "Fine tune part 1"
model.fit_generator(train_generator(train_datagen, train_df),
          nb_epoch=nb_epoch,
          samples_per_epoch=samples_per_epoch, #50000,
          verbose=1,
          validation_data=(valid_x, valid_y),
          callbacks=[early_stopping, model_checkpoint],
          )

Model creation... 
Fine tune part 1
Epoch 1/6
12672/12672 [==============================] - 699s - loss: 0.7935 - acc: 0.5701 - val_loss: 0.7761 - val_acc: 0.6506
Epoch 2/6
12672/12672 [==============================] - 699s - loss: 0.5619 - acc: 0.7173 - val_loss: 0.5811 - val_acc: 0.7529
Epoch 3/6
12672/12672 [==============================] - 699s - loss: 0.4304 - acc: 0.7874 - val_loss: 0.4376 - val_acc: 0.8370
Epoch 4/6
12672/12672 [==============================] - 700s - loss: 0.3432 - acc: 0.8281 - val_loss: 0.3501 - val_acc: 0.8648
Epoch 5/6
12672/12672 [==============================] - 699s - loss: 0.2703 - acc: 0.8634 - val_loss: 0.2622 - val_acc: 0.9174
Epoch 6/6
 5888/12672 [============>.................] - ETA: 363s - loss: 0.2236 - acc: 0.8830

KeyboardInterrupt: 

In [225]:
samples_per_epoch = 32*300
for layer in model.layers[38:]:
    layer.trainable = True
#model.optimizer.lr = 1e-5
nb_epoch = 1
print "Fine tune part 2"
model.fit_generator(train_generator(train_datagen, df=train_df),
          nb_epoch=nb_epoch,
          samples_per_epoch=samples_per_epoch,
          verbose=1,
          validation_data=(valid_x, valid_y),
          #callbacks=[model_checkpoint, early_stopping], # , 
          )

Fine tune part 2
Epoch 1/1
9600/9600 [==============================] - 536s - loss: 0.1845 - acc: 0.8997 - val_loss: 0.5499 - val_acc: 0.7946


In [226]:
test_preds_ls = []
for ii in range(3):
    test_preds_ls.append(model.predict_generator(test_generator(test_df, train_datagen), 
                                         val_samples = test_df.shape[0])) 
    model.fit_generator(train_generator(train_datagen, df=train_df),
              nb_epoch=1,
              samples_per_epoch=samples_per_epoch,
              verbose=1,
              validation_data=(valid_x, valid_y),
              #callbacks=[model_checkpoint, early_stopping], # , 
              )
test_preds_ls.append(model.predict_generator(test_generator(test_df, train_datagen), 
                                        val_samples = test_df.shape[0])) 

0
Epoch 1/1
9600/9600 [==============================] - 534s - loss: 0.1513 - acc: 0.9106 - val_loss: 0.1406 - val_acc: 0.9708
0
Epoch 1/1
9600/9600 [==============================] - 534s - loss: 0.1228 - acc: 0.9196 - val_loss: 0.1148 - val_acc: 0.9751
0
Epoch 1/1
9600/9600 [==============================] - 535s - loss: 0.1030 - acc: 0.9275 - val_loss: 0.0960 - val_acc: 0.9817
0


In [227]:
test_preds = sum(test_preds_ls)/len(test_preds_ls)

In [228]:
test_sub = pd.DataFrame(test_preds, columns=CERV_CLASSES)
test_sub['image_name'] = test_df['img'].str.split('/').apply(lambda x: x[-1])
test_sub = test_sub[['image_name'] + CERV_CLASSES ]
test_sub.head(3)

,image_name,Type_1,Type_2,Type_3
0,400.jpg,0.034559,0.262604,0.702837
1,430.jpg,0.059897,0.774324,0.165780
2,303.jpg,0.460207,0.525673,0.014120


In [230]:
import time
timestr = time.strftime("%Y%m%d")
if full:
    subm_name = '../sub/sub_dara_full_remove_addl_3xbag_' + timestr + '.csv' #'.csv.gz'
else:
    subm_name = '../sub/sub_dara_part_remove_addl_3xbag_' + timestr + '.csv' #'.csv.gz'
    
test_sub.to_csv(subm_name, index=False)#, compression='gzip')
FileLink(subm_name)

/home/ubuntu/cervical/sub/sub_dara_full_remove_addl_3xbag_20170605.csv